## Adapting to Your Use Case

To fully leverage the power of QB-RAG, you can prompt engineer the two prompts which are responsible for: question generation and answerability.

1. Customizing Prompts:
To modify the prompts, edit the converters.py file. You'll find two Prompt objects named `QUESTION_GENERATION_PROMPT` and `ANSWERABILITY_PROMPT`. Adjust the role/task and one/few-shot examples within these prompts to better fit your specific use case.

2. Controlling Question Generation:
You can also influence the number of questions generated from each chunk by implementing custom logic in the `_create_question_generation_prompt()` function. Currently, the default is set to generate 10 questions per chunk. Modify this logic to align with your requirements.

By tailoring these aspects, you can optimize the technique to better suit your needs.

## Loading the Secret Keys

Please ensure you've followed the instructions in the README to set up your `.env` file. If you prefer to use API keys directly rather than through a `.env` file, you'll need to incorporate them into your code manually as per your usual practice.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

## Setting Up the LLM

In this implementation, I have utilized OpenAI's GPT-3.5 Turbo as the language model (LLM). However, you are welcome to use any LLM of your choice, provided it is supported by LangChain or inherits from the `langchain_core.language_models.llms` class.

In [3]:
from langchain_openai.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = ChatOpenAI(model="gpt-3.5-turbo")

## Setting Up the Vector Database

In this project, Pinecone is used as the vector database. However, you can choose any vector database that is a there in Langchain (subclass of `langchain_core.vectorstores.base`).

Additionally, you have the flexibility to select the embedder for embedding questions. In this setup, I have utilized OpenAI's `text-embedding-3-small`. Adjust the embedder according to your requirements as needed.

In [5]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "qb-rag-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

## Loading the Data

Prepare a list of all the context/document text (LangChain lingo) that you have stored after the chunking process of your data source.

In this implementation, I am loading the chunks from a `text.file`.

In [13]:
def read_chunks_to_list(file_path):
    with open(file_path, "r") as file:
        file_content = file.read().strip()
        chunks = file_content.split("\n\n")
    return chunks


file_path = "text.txt"
chunks = read_chunks_to_list(file_path)

# Create a Converter Object and Loop

- **`questions_generated`**: This argument specifies how many questions will be generated from each context chunk or document text.
- **`max_retries`**: This argument sets the number of retry attempts in case of failure, such as when the output or response from the LLM does not match the expected format.
- **`reproducibility`**: This argument determines how many times the answerability of the generated questions will be tested. The results are aggregated using a voting mechanism to determine the final output.

In [15]:
from QB_RAG.converter import Converter

cvt = Converter(vector_store, llm, questions_generated=10, max_retries=1, reproducibility=1)

for i in chunks:
    cvt.add_documents(i)